<a href="https://colab.research.google.com/github/MatchLab-Imperial/deep-learning-course/blob/master/06_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursework

## Task 1: Non-linear Transformations for Representation Learning

PCA is a standard dimensionality reduction technique that uses a linear transformation. In this task we are going to define two autoencoders, one convolutional and one without using any convolutional layer, that are capable of learning a non-linear transformation to reduce the dimensionality of the input MNIST image, and we will compare those autoencoders to PCA. A way to evaluate the quality of the representations produced by both PCA and the autoencoder is to learn a classifier on top of those representations with reduced dimensionality. If the classifier has high accuracy, then the representations can be considered meaningful. In our case, we will use representations with dimensionality 10 and we will use those representations to train a linear classifier, which is defined in the code below.

The given example architectures for both the non-convolutional and the convolutional autoencoder already produce, after training them, representations of similar quality as PCA. Modify the given architectures and try to increase the accuracy when training a linear classifier on top of the autoencoder representations. The code given below may help you understand the pipeline.

As in past notebooks, treat the MNIST test set as your validation set. You can use any of the layers and techniques presented in past notebooks, the only constraints are that the non-convolutional autoencoder should not have any Conv2d layer, that the convolutional autoencoder should include Conv2d layers, and that the representation vector should have dimensionality 10.

**Report**:
* Table with the accuracy of `classifier` (defined below) obtained with the representations from your two proposed  autoencoder architectures (non-convolutional and convolutional autoencoder) and also with PCA with 10 components in the training set and the validation set. Additionally, include in the table the MSE error in both training and validation set for your non-convolutional autoencoder, for your convolutional autoencoder and for the PCA method. State clearly your two final autoencoder architectures and discuss the results.

We will use MNIST for this task. First, we resize all the images to have a resolution of 32x32, which will make the definition of the convolutional autoencoder easier.

In [ ]:
# Fix seed for reproducibility
set_seed(42)

# Load MNIST and preprocess: normalize and resize to 32x32 with 1 channel
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

def resize_dataset(dataset):
    imgs = []
    labels = []
    for img, label in dataset:
        # img shape (1,28,28), convert to numpy
        img_np = img.numpy().transpose(1,2,0)  # (28,28,1)
        img_resized = resize(img_np, (32,32,1), anti_aliasing=True).astype(np.float32)
        imgs.append(img_resized)
        labels.append(label)
    imgs = np.array(imgs)
    labels = np.array(labels)
    return imgs, labels

x_train_32, y_train = resize_dataset(train_dataset)
x_test_32, y_test = resize_dataset(test_dataset)

# Convert numpy arrays to torch tensors
x_train_32 = torch.tensor(x_train_32).permute(0,3,1,2)  # (N,1,32,32)
x_test_32 = torch.tensor(x_test_32).permute(0,3,1,2)
y_train = torch.tensor(y_train).long()
y_test = torch.tensor(y_test).long()

train_loader = DataLoader(TensorDataset(x_train_32, x_train_32), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_32, x_test_32), batch_size=128)

You can modify the code below to define your non-convolutional autoencoder. You can use any layer you want apart from `Conv2D` layers for this autoencoder.

In [ ]:
### Non-convolutional Autoencoder ###
class NonConvAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
            nn.Linear(32*32, 10),  # Representation with dimensionality 10
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 32*32),
            nn.Unflatten(1, (1, 32, 32)),
        )

    def forward(self, x):
        x = self.flatten(x)
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

    def encode(self, x):
        x = self.flatten(x)
        z = self.encoder(x)
        return z

In [ ]:
set_seed(42)

autoencoder = NonConvAutoencoder()
print(torchinfo.summary(autoencoder, input_size=(1, 1, 32, 32)))
print()

train(
    autoencoder,
    train_loader,
    nn.MSELoss(),
    optim.Adam(autoencoder.parameters()),
    num_epochs = 20,
)

You can modify the code below to define your convolutional autoencoder. For this autoencoder you need to include `Conv2D` layers in your design, but you can use any other layer too. We show an example of a simple convolutional architecture below

In [ ]:
### Convolutional Autoencoder ###
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),  # output: 32x16x16
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32*16*16, 10)  # representation 10 dim
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(10, 1*16*16),
            nn.Unflatten(1, (1,16,16)),
            nn.Upsample(scale_factor=2, mode='nearest'),  # upsample to 32x32
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

    def encode(self, x):
        return self.encoder(x)

In [ ]:
# You can modify the number of epochs or other hyperparameters
set_seed(42)

conv_autoencoder = ConvAutoencoder()
print(torchinfo.summary(conv_autoencoder, input_size=(1, 1, 32, 32)))
print()

train(
    conv_autoencoder,
    train_loader,
    nn.MSELoss(),
    optim.Adam(conv_autoencoder.parameters()),
    num_epochs = 20,
)

Below you have the code you will use to train the classifier. We first extract the representations using any of the two autoencoders we just trained or PCA and then we train the classifier on top, which is just a simple Dense layer. Better representations should make it easier for the given simple classifier to separate the classes and, therefore, have larger accuracy.

In [ ]:
### PCA ###
pca = PCA(n_components=10)
pca.fit(x_train_32.numpy().reshape(len(x_train_32), -1))

## We compute the representations for the different methods
representation_pca_train = pca.transform(x_train_32.numpy().reshape(len(x_train_32), -1))
representation_pca_test = pca.transform(x_test_32.numpy().reshape(len(x_test_32), -1))

# predict_representation is defined at the beginning of this notebook
representation_auto_train = predict_representation(autoencoder, x_train_32)
representation_auto_test = predict_representation(autoencoder, x_test_32)
representation_conv_auto_train = predict_representation(conv_autoencoder, x_train_32)
representation_conv_auto_test = predict_representation(conv_autoencoder, x_test_32)

# Compute PCA reconstruction MSE
reconst_train = pca.inverse_transform(representation_pca_train).reshape(-1,1,32,32)
train_mse_pca = ((reconst_train - x_train_32.numpy())**2).mean()

reconst_test = pca.inverse_transform(representation_pca_test).reshape(-1,1,32,32)
test_mse_pca = ((reconst_test - x_test_32.numpy())**2).mean()

# We print the MSE for PCA, which you need to include on the table
print(f'PCA Train MSE: {train_mse_pca:.4f}')
print(f'PCA Test MSE: {test_mse_pca:.4f}')

In [ ]:
### Linear classifier to test representation quality, DO NOT MODIFY IT!
class LinearClassifier(nn.Module):
    def __init__(self, input_dim=10, num_classes=10):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

In [ ]:
def train_classifier(X_train, y_train, X_val, y_val, epochs=30):
    set_seed(42)

    X_train = torch.tensor(X_train).float()
    X_val = torch.tensor(X_val).float()

    train_ds = TensorDataset(X_train, y_train)
    val_ds = TensorDataset(X_val, y_val)

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=128)

    model = LinearClassifier(input_dim=X_train.shape[1], num_classes=10).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

        # Validation accuracy
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                preds = torch.argmax(logits, dim=1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)
        acc = correct / total

        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1}/{epochs}] Validation Accuracy: {acc*100:.2f}%")

    return acc

print("Training classifier on PCA representations")
acc_pca = train_classifier(representation_pca_train, y_train, representation_pca_test, y_test)
print()

print("Training classifier on Non-Convolutional Autoencoder representations")
acc_auto = train_classifier(representation_auto_train, y_train, representation_auto_test, y_test)
print()

print("Training classifier on Convolutional Autoencoder representations")
acc_conv_auto = train_classifier(representation_conv_auto_train, y_train, representation_conv_auto_test, y_test)
print()

The code below can help you visualize the quality of the learnt representations. tSNE is a dimensionality reduction technique that leads to nice plots, so we reduce the representations of dimensionality 10 we just learnt to dimensionality 2 via tSNE and plot it. You do not have to include the figures in the report, it is just a qualitative way for you to see the quality of your representations.

In [ ]:
## We use tSNE for our dimensionality reduction technique so we can
## plot the features using a 2D plot as it leads to nice plots.
## However, tSNE is tricky to use as a general dimensionality reduction method
## for clustering due to issues mentioned here: https://distill.pub/2016/misread-tsne/
## TSNE: https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding
## Nice article explaining shortcomings: https://distill.pub/2016/misread-tsne/

## Use these parameters, the plots are highly dependent on perplexity value
tsne = TSNE(n_components=2, verbose=1, perplexity=40, max_iter=500, n_jobs=-1)
representation_tsne = tsne.fit_transform(representation_auto_test)
plot_representation_label(representation_tsne, y_test)

You can also check how the reconstructed images look with the autoencoders you just trained.

In [ ]:
ind = np.random.randint(x_test.shape[0] -  1)
## The function below is defined in the tutorial
plot_recons_original(np.expand_dims(x_test_32[ind],0), y_test[ind], conv_autoencoder, size_image=(32,32))

## Task 2: Custom Loss Functions

In Image-to-Image tasks, researchers have found that an approach to improve the robustness of autoencoders is to replace the quadratic error with a loss function that is more robust to outliers.

There is a lot of interest in defining which loss function helps the most specific tasks. For instance, super-image resolution and image denoising may have different optimum loss functions, even though, they are trained in a very similar manner. Therefore, in this task, we will focus on using multiple loss functions to train an image denoise model. Sometimes, you may need to use a loss function that is not defined in Keras. If that happens, you can define it yourself and use it in the model.compile() module. We explain now how to do that.

You must define some variables:

* **True values** are those that we are aiming to generate, e.g. GT images.
* **Predicted values** are those that the network has generated,  e.g. denoised images.
* **Loss value** is the computed loss between true and predicted values,  e.g. MSE value.

The common structure for the custom loss method is as follows:

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, predicted, target):
        return []

We are going to use TinyImagenet, and add synthetic noise as before. This task requires a lot of RAM, thus, before starting it, please clean your RAM memory by restarting the Colab session.

In [ ]:
# download TinyImageNet
! git clone https://github.com/seshuad/IMagenet

In [ ]:
class TinyImageNetNoisyDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, noise_std=0.2):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.noise_std = noise_std
        self.image_files = []

        if self.split == 'train':
            # For training data, images are in subfolders per class
            wnids_path = os.path.join(root_dir, 'tiny-imagenet-200', 'wnids.txt')
            with open(wnids_path, 'r') as f:
                classes = [line.strip() for line in f]
            for cls in classes:
                image_dir = os.path.join(root_dir, 'tiny-imagenet-200', split, cls, 'images')
                for img_file in os.listdir(image_dir):
                    if img_file.endswith('.JPEG'):
                        self.image_files.append(os.path.join(image_dir, img_file))

        elif self.split == 'val':
            # For validation data, images are in a single folder and annotations are in a file
            annotations_path = os.path.join(root_dir, 'tiny-imagenet-200', split, 'val_annotations.txt')
            with open(annotations_path, 'r') as f:
                for line in f:
                    img_name, _, _, _, _, _ = line.strip().split('\t')
                    self.image_files.append(os.path.join(root_dir, 'tiny-imagenet-200', split, 'images', img_name))

        else:
            raise ValueError(f"Invalid split: {split}. Use 'train' or 'val'.")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        noisy_img = img + torch.randn_like(img) * self.noise_std
        noisy_img = torch.clamp(noisy_img, 0, 1)

        return noisy_img, img  # return noisy input and clean target

# Image transformations (resizing and normalization)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Corrected root_dir to point to the base directory of TinyImageNet
train_dataset = TinyImageNetNoisyDataset('./IMagenet', transform=transform)
test_dataset = TinyImageNetNoisyDataset('./IMagenet', split='val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)

We are going to use the UNet architecture for this task, however, you could use any autoencoder of your choice.

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Encoder
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv5 = nn.Conv2d(512, 1024, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # Decoder
        self.up6 = nn.Conv2d(1024, 512, 3, padding=1)
        self.up7 = nn.Conv2d(512, 256, 3, padding=1)
        self.up8 = nn.Conv2d(256, 128, 3, padding=1)
        self.up9 = nn.Conv2d(128, 64, 3, padding=1)

        self.up_sample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv_final = nn.Conv2d(64, 3, 3, padding=1)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # Encoder
        c1 = self.relu(self.conv1(x))
        p1 = self.pool(c1)

        c2 = self.relu(self.conv2(p1))
        p2 = self.pool(c2)

        c3 = self.relu(self.conv3(p2))
        p3 = self.pool(c3)

        c4 = self.relu(self.conv4(p3))
        d4 = self.dropout(c4)
        p4 = self.pool(d4)

        c5 = self.relu(self.conv5(p4))
        d5 = self.dropout(c5)

        # Decoder
        up6 = self.up_sample(d5)
        up6 = self.relu(self.up6(up6))
        merge6 = torch.cat([d4, up6], dim=1)
        c6 = self.relu(self.up6(merge6))

        up7 = self.up_sample(c6)
        up7 = self.relu(self.up7(up7))
        merge7 = torch.cat([c3, up7], dim=1)
        c7 = self.relu(self.up7(merge7))

        up8 = self.up_sample(c7)
        up8 = self.relu(self.up8(up8))
        merge8 = torch.cat([c2, up8], dim=1)
        c8 = self.relu(self.up8(merge8))

        up9 = self.up_sample(c8)
        up9 = self.relu(self.up9(up9))
        merge9 = torch.cat([c1, up9], dim=1)
        c9 = self.relu(self.up9(merge9))

        output = self.conv_final(c9)
        output = torch.sigmoid(output)  # constrain output to [0,1]

        return output

Now, an example of how to train UNet architecture with a custom MSE loss function:

In [ ]:
class CustomMSELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, predicted, target):
        return torch.mean((predicted - target) ** 2)

In [ ]:
set_seed(42)

model = UNet()
print(torchinfo.summary(model, input_size=(1, 3, 64, 64)))
print()

train(
    model,
    train_loader,
    CustomMSELoss(),
    optim.Adam(model.parameters(), lr=1e-4),
    num_epochs = 10,
)
print()

evaluate(model, test_loader, CustomMSELoss())

Use the following code for visualising examples:

In [ ]:
def show_noisy_denoised_clean(idx=0):
    model.eval()
    noisy_img, clean_img = test_dataset[idx]
    noisy_img_t = noisy_img.unsqueeze(0).to(device)

    with torch.no_grad():
        denoised_img = model(noisy_img_t).cpu().squeeze(0)

    # Convert tensors to numpy arrays for plotting
    noisy_np = noisy_img.permute(1, 2, 0).numpy()
    denoised_np = denoised_img.permute(1, 2, 0).numpy()
    clean_np = clean_img.permute(1, 2, 0).numpy()

    # Plot side by side
    fig, axs = plt.subplots(1, 3, figsize=(12,4))
    axs[0].imshow(np.clip(noisy_np, 0, 1))
    axs[0].set_title('Noisy Input')
    axs[0].axis('off')
    axs[1].imshow(np.clip(denoised_np, 0, 1))
    axs[1].set_title('Denoised Output')
    axs[1].axis('off')
    axs[2].imshow(np.clip(clean_np, 0, 1))
    axs[2].set_title('Clean Target')
    axs[2].axis('off')
    plt.show()

show_noisy_denoised_clean()

Try to use a different loss function and see which one gives you the best result. Some well-known loss functions for image denoising are:

*  Structural Similarity Index ([SSIM](https://en.wikipedia.org/wiki/Structural_similarity))
*  Multiscale Structural Similarity Index ([MS-SSIM](https://ieeexplore.ieee.org/document/1292216?arnumber=1292216&tag=1))
* 1 / [PSNR](https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio)
* MAE
* [L0](https://arxiv.org/pdf/1803.04189.pdf)

Check the [Noise2Noise](https://arxiv.org/pdf/1803.04189.pdf) paper to learn more about alternative losses.

**Report:**
*  In this task, you are asked to build a table containing the MSE results on the test split of models trained with different loss functions. Use two or three different loss functions from the previous list and discuss the differences you observe. Report denoised images to support your arguments. You may need to modify the UNet model definition to use some of the previous losses.

Use the following code and write your custom loss function within the provided `CustomLoss` class.



In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, predicted, target):
        '''
        Define your loss here
        '''
        return 0